In [1]:
! python utils/preprocess_text_word_vectors.py

Loading vectors from data/combined.vector
Found 9493 vectors in data/combined.vector
Processed 9000 vectors (94.81 %)
Saving word vectors in data/word2vec.en.npy



In [2]:
import cPickle
train_dataset = cPickle.load(open('data/webnlg/train_data.pickle', 'rb'))
dev_dataset = cPickle.load(open('data/webnlg/dev_data.pickle', 'rb'))
test_dataset = cPickle.load(open('data/webnlg/test_data.pickle', 'rb'))
test_unseen_dataset = cPickle.load(open('data/webnlg/test_unseen.pickle', 'rb'))
test_gkb_dataset = cPickle.load(open('data/gkb/test_data.pickle', 'rb'))

''' ADDITIONAL '''
#gkb_dataset = cPickle.load(open('data/gkb/train_data.pickle', 'rb'))
dev_dataset = dev_dataset[:200]
train_dataset += dev_dataset[200:]
#train_dataset += gkb_dataset
''' ADDITIONAL '''
print

In [3]:
import utility
from kitchen.text.converters import getwriter, to_bytes, to_unicode
from kitchen.i18n import get_translation_object
translations = get_translation_object('example')
_ = translations.ugettext
b_ = translations.lgettext

In [4]:
encoder_model = 'GLSTM' # BLSTM, TLSTM, TCNN, GLSTM
includeLabel = True # delexicalization
triple_mode = 'graph' # single, graph
auto_adaptive = True # attention model

if triple_mode == 'single':
    triple_max_len = 20
    input_max_len = 140
    output_max_len = 70
elif triple_mode == 'graph':
    triple_max_len = 5
    input_max_len = 140
    output_max_len = 70

In [5]:
utility.createDataset(train_dataset, "train", triple_mode, includeLabel, triple_max_len, 'w')
utility.createDataset(dev_dataset, "dev", triple_mode, includeLabel, triple_max_len, 'a')
utility.createDataset(test_dataset, "test", triple_mode, includeLabel, triple_max_len, 'a')
utility.createDataset(test_unseen_dataset, "test_unseen", triple_mode, includeLabel, triple_max_len, 'a')
utility.createDataset(test_gkb_dataset, "test_gkb", triple_mode, includeLabel, triple_max_len, 'a')

In [6]:
from keras_wrapper.dataset import Dataset, saveDataset
from data_engine.prepare_data import keep_n_captions
ds = Dataset('GKB_Data', 'gkb', silence=False)

Using Theano backend.
Using cuDNN version 6021 on context None
Mapped name None to device cuda0: Tesla K40c (0000:03:00.0)


In [7]:
ds.setInput('data/vocab.used',
            'train',
            type='text',
            id='all_vocab',
            required=False,
            tokenization='tokenize_none',
            pad_on_batch=False,
            build_vocabulary=True,
            offset=1,
            fill='end',
            max_text_len=output_max_len,
            max_words=30000)
ds.setOutput('data/train.tar',
             'train',
             type='text',
             id='target_text',
             tokenization='tokenize_none',
             build_vocabulary='all_vocab',
             pad_on_batch=False,
             sample_weights=True,
             max_text_len=output_max_len,
             max_words=30000,
             min_occ=0)

ds.setOutput('data/dev.tar',
             'val',
             type='text',
             id='target_text',
             pad_on_batch=False,
             tokenization='tokenize_none',
             sample_weights=True,
             max_text_len=output_max_len,
             max_words=0)

ds.setInput('data/train.src',
            'train',
            type='text',
            id='source_text',
            pad_on_batch=False,
            tokenization='tokenize_none',
            build_vocabulary='all_vocab',
            fill='end',
            max_text_len=input_max_len,
            max_words=30000,
            min_occ=0)
ds.setInput('data/dev.src',
            'val',
            type='text',
            id='source_text',
            pad_on_batch=False,
            tokenization='tokenize_none',
            fill='end',
            max_text_len=input_max_len,
            min_occ=0)

ds.setInput('data/train.tar',
            'train',
            type='text',
            id='state_below',
            required=False,
            tokenization='tokenize_none',
            pad_on_batch=False,
            build_vocabulary='all_vocab',
            offset=1,
            fill='end',
            max_text_len=output_max_len,
            max_words=30000)
ds.setInput(None,
            'val',
            type='ghost',
            id='state_below',
            required=False)

keep_n_captions(ds, repeat=1, n=1, set_names=['val'])

ds.vocabulary['source_text'] = ds.vocabulary['all_vocab']
ds.vocabulary['state_below'] = ds.vocabulary['all_vocab']
ds.vocabulary['target_text'] = ds.vocabulary['all_vocab']

ds.vocabulary_len['source_text'] = ds.vocabulary_len['all_vocab']
ds.vocabulary_len['state_below'] = ds.vocabulary_len['all_vocab']
ds.vocabulary_len['target_text'] = ds.vocabulary_len['all_vocab']

saveDataset(ds, 'datasets')

[01/10/2019 18:44:06] 	Applying tokenization function: "tokenize_none".
[01/10/2019 18:44:07] Creating vocabulary for data with id 'all_vocab'.
[01/10/2019 18:44:09] 	 Total: 9516 unique words in 83124 sentences with a total of 2434451 words.
[01/10/2019 18:44:09] Creating dictionary of 30000 most common words, covering 100.0% of the text.
[01/10/2019 18:44:09] Loaded "train" set inputs of type "text" with id "all_vocab" and length 0.
[01/10/2019 18:44:09] 	Applying tokenization function: "tokenize_none".
[01/10/2019 18:44:09] 	Reusing vocabulary named "all_vocab" for data with id "target_text".
[01/10/2019 18:44:09] Loaded "train" set outputs of type "text" with id "target_text" and length 34653.
[01/10/2019 18:44:09] 	Applying tokenization function: "tokenize_none".
[01/10/2019 18:44:09] Loaded "val" set outputs of type "text" with id "target_text" and length 200.
[01/10/2019 18:44:09] 	Applying tokenization function: "tokenize_none".
[01/10/2019 18:44:10] 	Reusing vocabulary named "

In [8]:
from config import load_parameters
from model_zoo import TranslationModel
import utils
from keras_wrapper.cnn_model import loadModel
from keras_wrapper.dataset import loadDataset
from keras_wrapper.extra.callbacks import PrintPerformanceMetricOnEpochEndOrEachNUpdates
params = load_parameters()
dataset = loadDataset('datasets/Dataset_GKB_Data.pkl')

params['INPUT_VOCABULARY_SIZE'] = dataset.vocabulary_len['all_vocab']
params['OUTPUT_VOCABULARY_SIZE'] = dataset.vocabulary_len['all_vocab']
params['INPUT_MAX_LEN'] = input_max_len
params['OUTPUT_MAX_LEN'] = output_max_len
params['TRIPLE_MAX_LEN'] = triple_max_len
params['ENCODER_MODEL'] = encoder_model
params['AUTO_ADAPTIVE'] = auto_adaptive

nmt_model = TranslationModel(params,
                             model_type='GroundHogModel', 
                             model_name='tutorial_model',
                             vocabularies=dataset.vocabulary,
                             store_path='trained_models/gkb_model/',
                             verbose=True)

[01/10/2019 18:44:10] Log file (/home/bayu/.picloud/cloud.log) opened
[01/10/2019 18:44:11] <<< Loading Dataset instance from datasets/Dataset_GKB_Data.pkl ... >>>
[01/10/2019 18:44:11] <<< Dataset instance loaded >>>


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
source_text (InputLayer)        (None, 140)          0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 5)            0           source_text[0][0]                
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, 5)            0           source_text[0][0]                
__________________________________________________________________________________________________
source_word_embedding (Embeddin (None, 5, 300)       2855700     lambda_2[0][0]                   
                                                                 lambda_4[0][0]                   
          

target_word_embedding (Embeddin (None, None, 300)    2855700     state_below[0][0]                
__________________________________________________________________________________________________
initial_state (Dense)           (None, 512)          524800      masked_mean_1[0][0]              
__________________________________________________________________________________________________
initial_memory (Dense)          (None, 512)          524800      masked_mean_1[0][0]              
__________________________________________________________________________________________________
state_below_gaussian_noise (Gau (None, None, 300)    0           target_word_embedding[0][0]      
__________________________________________________________________________________________________
initial_state_gaussian_noise (G (None, 512)          0           initial_state[0][0]              
__________________________________________________________________________________________________
initial_me

target_text (TimeDistributed)   (None, None, 9519)   2865219     out_layer_linear_0_dropout[0][0] 
Total params: 28,324,281
Trainable params: 28,317,185
Non-trainable params: 7,096
__________________________________________________________________________________________________


In [9]:
inputMapping = dict()
for i, id_in in enumerate(params['INPUTS_IDS_DATASET']):
    pos_source = dataset.ids_inputs.index(id_in)
    id_dest = nmt_model.ids_inputs[i]
    inputMapping[id_dest] = pos_source
nmt_model.setInputsMapping(inputMapping)

outputMapping = dict()
for i, id_out in enumerate(params['OUTPUTS_IDS_DATASET']):
    pos_target = dataset.ids_outputs.index(id_out)
    id_dest = nmt_model.ids_outputs[i]
    outputMapping[id_dest] = pos_target
nmt_model.setOutputsMapping(outputMapping)

extra_vars = {'language': 'en',
              'n_parallel_loaders': 8,
              'tokenize_f': eval('dataset.' + 'tokenize_none'),
              'beam_size': 12,
              'maxlen': output_max_len,
              'model_inputs': ['source_text', 'state_below'],
              'model_outputs': ['target_text'],
              'dataset_inputs': ['source_text', 'state_below'],
              'dataset_outputs': ['target_text'],
              'normalize': True,
              'alpha_factor': 0.6,
              'val': {'references': dataset.extra_variables['val']['target_text']}
              }

vocab = dataset.vocabulary['all_vocab']['idx2words']
callbacks = []
callbacks.append(PrintPerformanceMetricOnEpochEndOrEachNUpdates(nmt_model,
                                                                dataset,
                                                                gt_id='target_text',
                                                                metric_name=['coco'],
                                                                set_name=['val'],
                                                                batch_size=64,
                                                                each_n_epochs=1,
                                                                extra_vars=extra_vars,
                                                                reload_epoch=0,
                                                                is_text=True,
                                                                index2word_y=vocab,
                                                                sampling_type='max_likelihood',
                                                                beam_search=True,
                                                                save_path=nmt_model.model_path,
                                                                start_eval_on_epoch=0,
                                                                write_samples=True,
                                                                write_type='list',
                                                                verbose=True))

training_params = {'n_epochs': 50,
                   'batch_size': 64,
                   'maxlen': output_max_len,
                   'epochs_for_save': 1,
                   'verbose': 0,
                   'eval_on_sets': [], 
                   'n_parallel_loaders': 16,
                   'extra_callbacks': callbacks,
                   'reload_epoch': 0,
                   'epoch_offset': 0}

In [10]:
nmt_model.trainNet(dataset, training_params)

[01/10/2019 18:44:26] WARNING: parallel loaders are not implemented
ERROR (theano.gof.opt): SeqOptimizer apply <theano.scan_module.scan_opt.PushOutScanOutput object at 0x7feb65de52d0>
[01/10/2019 18:44:38] SeqOptimizer apply <theano.scan_module.scan_opt.PushOutScanOutput object at 0x7feb65de52d0>
ERROR (theano.gof.opt): Traceback:
[01/10/2019 18:44:38] Traceback:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/home/bayu/anaconda2/envs/gtrlstm_extension/lib/python2.7/site-packages/theano/gof/opt.py", line 251, in apply
    sub_prof = optimizer.optimize(fgraph)
  File "/home/bayu/anaconda2/envs/gtrlstm_extension/lib/python2.7/site-packages/theano/gof/opt.py", line 97, in optimize
    ret = self.apply(fgraph, *args, **kwargs)
  File "/home/bayu/anaconda2/envs/gtrlstm_extension/lib/python2.7/site-packages/theano/scan_module/scan_opt.py", line 685, in apply
    node = self.process_node(fgraph, node)
  File "/home/bayu/anaconda2/envs/gtrlstm_extension/lib/python2.7/site-p


Sampling 200/200  -  ETA: 3s   
 Total cost of the translations: 2655.928583 	 Average cost of the translations: 13.279643
The sampling took: 606.835282 secs (Speed: 3.034176 sec/sample)


[01/10/2019 19:10:04] Prediction output 0: target_text (text)
[01/10/2019 19:10:04] Decoding beam search prediction ...
[01/10/2019 19:10:04] Evaluating on metric coco


[01/10/2019 19:10:14] Computing coco scores on the val split...
[01/10/2019 19:10:14] Bleu_1: 0.446417650261
[01/10/2019 19:10:14] Bleu_2: 0.256409387209
[01/10/2019 19:10:14] Bleu_3: 0.154498678467
[01/10/2019 19:10:14] Bleu_4: 0.099515784306
[01/10/2019 19:10:14] CIDEr: 0.702420428765
[01/10/2019 19:10:14] METEOR: 0.303670072197
[01/10/2019 19:10:14] ROUGE_L: 0.440470827673
[01/10/2019 19:10:14] TER: 0.700119474313
[01/10/2019 19:10:14] Done evaluating on metric coco
[01/10/2019 19:10:14] <<< Progress plot saved in trained_models/gkb_model//epoch_1.jpg >>>
[01/10/2019 19:10:14] <<< Saving model to trained_models/gkb_model//epoch_1 ... >>>
[01/10/2019 19:10:14] <<< Saving model_init to trained_models/gkb_model//epoch_1_structure_init.json... >>>
[01/10/2019 19:10:14] <<< Saving model_next to trained_models/gkb_model//epoch_1_structure_next.json... >>>
[01/10/2019 19:10:14] <<< Model saved >>>
[01/10/2019 19:10:14] <<< Saving model to trained_models/gkb_model//epoch_1 ... >>>
[01/10/20

[01/10/2019 19:10:15] <<< Saving model_next to trained_models/gkb_model//epoch_1_structure_next.json... >>>
[01/10/2019 19:10:17] <<< Model saved >>>
[01/10/2019 19:23:30] WARNING: parallel loaders are not implemented
[01/10/2019 19:23:30] <<< Predicting outputs of val set >>>


...


Sampling 200/200  -  ETA: 2s   
 Total cost of the translations: 468.962443 	 Average cost of the translations: 2.344812
The sampling took: 568.697257 secs (Speed: 2.843486 sec/sample)


[02/10/2019 13:26:03] Prediction output 0: target_text (text)
[02/10/2019 13:26:03] Decoding beam search prediction ...
[02/10/2019 13:26:03] Evaluating on metric coco


[02/10/2019 13:26:13] Computing coco scores on the val split...
[02/10/2019 13:26:13] Bleu_1: 0.693484654013
[02/10/2019 13:26:13] Bleu_2: 0.536263541533
[02/10/2019 13:26:13] Bleu_3: 0.434216711615
[02/10/2019 13:26:13] Bleu_4: 0.363044046851
[02/10/2019 13:26:13] CIDEr: 2.98936167149
[02/10/2019 13:26:13] METEOR: 0.431860250306
[02/10/2019 13:26:13] ROUGE_L: 0.604226751172
[02/10/2019 13:26:13] TER: 0.503285543608
[02/10/2019 13:26:13] Done evaluating on metric coco
[02/10/2019 13:26:13] <<< Progress plot saved in trained_models/gkb_model//epoch_43.jpg >>>
[02/10/2019 13:26:13] <<< Saving model to trained_models/gkb_model//epoch_43 ... >>>
[02/10/2019 13:26:14] <<< Saving model_init to trained_models/gkb_model//epoch_43_structure_init.json... >>>
[02/10/2019 13:26:14] <<< Saving model_next to trained_models/gkb_model//epoch_43_structure_next.json... >>>
[02/10/2019 13:26:14] <<< Model saved >>>
[02/10/2019 13:26:14] <<< Saving model to trained_models/gkb_model//epoch_43 ... >>>


[02/10/2019 13:26:14] <<< Saving model_init to trained_models/gkb_model//epoch_43_structure_init.json... >>>
[02/10/2019 13:26:15] <<< Saving model_next to trained_models/gkb_model//epoch_43_structure_next.json... >>>
[02/10/2019 13:26:15] <<< Model saved >>>


KeyboardInterrupt: 